# Emission RAG Architecture Project

## Install Necessary Modules

In [ ]:
pip install sentence-transformers
pip install duckdb

In [ ]:
pip install duckdb

## Import Necessary Libraries

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import duckdb

## Import Data

### Import downloaded carbon emission data in csv format, upload to postgres

In [ ]:
df = pd.read_csv('owid-co2-data.csv')

#filter only the latest data
filtered_df = df[df['year'] > 2000]
filtered_df.head(5)

In [ ]:
import os
import config  # Import the config file

# Set environment variables using values from config.py
os.environ["POSTGRES_HOST"] = config.POSTGRES_HOST
os.environ["POSTGRES_DB"] = config.POSTGRES_DB
os.environ["POSTGRES_USER"] = config.POSTGRES_USER
os.environ["POSTGRES_PASSWORD"] = config.POSTGRES_PASSWORD
os.environ["POSTGRES_SCHEMA"] = config.POSTGRES_SCHEMA

In [ ]:
print(os.getenv("POSTGRES_HOST"))  # Should print: your_postgres_host
print(os.getenv("POSTGRES_DB"))    # Should print: your_database_name
# print(os.getenv("POSTGRES_PASSWORD"))    # You Should never print: passwords or sensitive data

In [ ]:
# Environment variables for PostgreSQL connection
POSTGRES_HOST = os.getenv("POSTGRES_HOST", "localhost")
POSTGRES_DB = os.getenv("POSTGRES_DB", "postgres")
POSTGRES_USER = os.getenv("POSTGRES_USER", "postgres")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD", "postgres")
POSTGRES_PORT = os.getenv("POSTGRES_PORT", 5432) # leave at the default port
POSTGRES_SCHEMA = os.getenv("POSTGRES_SCHEMA", "public")

In [ ]:
import os
import sqlalchemy
from sqlalchemy import create_engine

# Create PostgreSQL connection string
pg_conn_string = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the SQLAlchemy engine
pg_engine = create_engine(pg_conn_string, connect_args={"options": f"-c search_path={POSTGRES_SCHEMA}"})


In [ ]:
schema_name = POSTGRES_SCHEMA
table_name = f"{schema_name}.example_table_4"

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
id SERIAL PRIMARY KEY,
example_column VARCHAR(255) NOT NULL);
"""

In [ ]:
drop_table_query = f"""
DROP TABLE "carbon_data"

"""

# DROP TABLE "sales"
# DROP TABLE "customers"
# DROP TABLE "products"

In [ ]:
import psycopg2
import pandas as pd
try:
    conn = psycopg2.connect(
        host=POSTGRES_HOST,
        dbname=POSTGRES_DB,
        user=POSTGRES_USER ,
        password=POSTGRES_PASSWORD,
        port=POSTGRES_PORT
    )    
    print("Connection successful!")
    
    # Set the schema if required
    with conn.cursor() as cur:
        cur.execute(f"SET search_path TO {POSTGRES_SCHEMA};")
        print(f"Schema set to {POSTGRES_SCHEMA}")

        # cur = conn.cursor()
        # cur.execute(create_table_query)
        cur.execute(drop_table_query)
        conn.commit()
        cur.close()

        print(f"Table {table_name} created successfully.")

except Exception as e:
    print("Error connecting to the database:", e)
finally:
    if 'conn' in locals() and conn:
        conn.close()
        print("Connection closed.")

In [ ]:
pg_engine = create_engine("postgresql://muhammad.tribosnia.24@ucl.ac.uk:rzMwmH@uclba-de25v2.cluster-cowglvndjvxv.eu-west-2.rds.amazonaws.com:5432/postgres")

filtered_df.to_sql("carbon_data", pg_engine, schema ='schema_muhammadtribosnia24uclacuk', index = False)


### Acess newsdata.io to get latest news, use duckdb to store in in parquet file

In [ ]:
import os
import config  # Import the config file
import requests

# Set environment variables using values from config.py
os.environ["API_KEY"] = config.newsio_api


# Environment variables for API connection
API_KEY = os.getenv("API_KEY")

In [ ]:
query = "carbon emissions"
url = f"https://newsdata.io/api/1/news?apikey={API_KEY}&q={query}&language=en"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    articles = data.get('results', [])[:5]  # Select only the first 5 articles
    for i, article in enumerate(articles, start=1):
        print(f"{i}. {article.get('title')}")
        print(f"   {article.get('link')}")
        print(f"   Source: {article.get('source_id')} | Published: {article.get('pubDate')}\n")
else:
    print("Error:", response.status_code, response.text)


In [ ]:
# Build the DataFrame
results = data['results']
df_news = pd.DataFrame([{
    'title': item.get('title'),
    'description': item.get('description'),
    'source': item.get('source_name'),
    'country': ', '.join(item.get('country', [])),
    'link': item.get('link'),
    'pubDate': item.get('pubDate')
} for item in results])

# Show the first 5 rows
df_news

In [ ]:
import pandas as pd
df_news = pd.read_csv('news.csv')
df_news = df_news.drop(columns=["Unnamed: 0"], errors='ignore')
df_news

In [ ]:
# 1. Save the DataFrame as a Parquet file
df_news.to_parquet("emission_news.parquet", index=False)
df_news

# 2. Connect to (or create) the DuckDB database
con = duckdb.connect("emission_news_data.duckdb")

# 3. Load the Parquet file into the DuckDB database as a table
con.execute("""
    CREATE OR REPLACE TABLE emission_news AS
    SELECT * FROM read_parquet('emission_news.parquet')
""")




In [ ]:
# Optional: Verify the data
con.execute("SELECT * FROM emission_news LIMIT 5").fetchdf()

## Creating RAG Summary and Sentence Embeddings Pipeline

### Download carbon data files from postgres and emission-related news from duckdb

In [ ]:
###carbon data
from sqlalchemy import text
pg_engine = create_engine("postgresql://muhammad.tribosnia.24@ucl.ac.uk:rzMwmH@uclba-de25v2.cluster-cowglvndjvxv.eu-west-2.rds.amazonaws.com:5432/postgres")

query = text("SELECT * FROM schema_muhammadtribosnia24uclacuk.carbon_data;")
# sales_data = pd.read_sql_query(query, conn)
df = pd.read_sql_query(query, pg_engine)
df

In [ ]:
###emission-related news
df_news = con.execute("SELECT * FROM emission_news").fetchdf()
df_news

### Turn dataframe into RAG Summary

In [ ]:
# Create rag_summary using all columns
df['rag_summary'] = df.apply(lambda row: (
    "[TYPE: CARBON DATA]\n"
    "Document: Full Carbon Data Entry\n" +
    "\n".join([f"{col}: {row[col]}" for col in df.columns if col != 'rag_summary' and col != 'embeddings']) +
    "\nSource: Structured Carbon Dataset"
), axis=1)

df

In [ ]:
df['rag_summary'][0]

In [ ]:
df_news['rag_summary'] = df_news.apply(lambda row: (
    "[TYPE: NEWS ARTICLE]\n"
    "Document: Climate & Emissions News\n"
    f"Title: {row['title']}\n"
    f"Description: {row['description']}\n"
    f"Source: {row['source']}\n"
    f"Country: {row['country']}\n"
    f"Published At: {row['pubDate']}\n"
    f"Link: {row['link']}\n"
    "Source: External News Feed"
), axis=1)

df_news



In [ ]:
df_news['rag_summary'][0]

### Turn RAG summary into sentence embeddings, and merge the RAG summary and the sentence embeddings from both dataset into a single dataset

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Step 2: Load SentenceTransformer Model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Generate Embeddings
carbondata_embeddings = embedder.encode(df["rag_summary"].tolist(), show_progress_bar=True)
news_embeddings = embedder.encode(df_news["rag_summary"].tolist(), show_progress_bar=True)

# Step 4: Store embeddings in a new column
df["embeddings"] = carbondata_embeddings.tolist()
df_news["embeddings"] = news_embeddings.tolist()

In [ ]:
### Merge RAG summary and embeddings
df_combined = pd.concat([
    df[['rag_summary', 'embeddings']],        # your carbon data
    df_news[['rag_summary', 'embeddings']]    # your news data
], ignore_index=True)

df_combined

In [ ]:
import pandas as pd
import pandas as np

In [ ]:
df_combined.to_csv('prepared_data.csv')

### Upload the Combined Data to Cloud Database (now use duckdb for testing)

In [ ]:
import duckdb

# 1. Connect to (or create) the DuckDB database
con = duckdb.connect("combined_emission_data.duckdb")

# 2. Register the Pandas DataFrame as a temporary view
con.register("df_combined_view", df_combined)

# 3. Create a permanent table from that view
con.execute("""
    CREATE OR REPLACE TABLE combined_data AS
    SELECT * FROM df_combined_view
""")

con.close()
